Читаю файл с твитами и разбиваю его на строки.

In [3]:
import json

twitter = []
for line in open('hw_3_twitter.json'):
    twitter.append(json.loads(line))

# Пункт 1

Чтобы найти количество твитов в наборе, нахожу длину получившегося файла, так как одна строка - один твит.

In [4]:
all_tweets = len(twitter)
print('Ответ: количество твитов в наборе -', all_tweets, 'твитов.')

Ответ: количество твитов в наборе - 2556 твитов.


# Пункт 2

Чтобы найти количество удалённых твитов, вытаскиваю список ключей каждого твита и проверяю первый элемент на совпадение со строкой "delete", так как "delete" - первый ключ удалённых твитов. 

In [5]:
deleted_counter = 0

for line in twitter:
    keys = list(line.keys())
    if keys[0] == 'delete':
        deleted_counter += 1

# делю кол-во удалённых твитов на кол-во всех твитов
deleted_percent = deleted_counter / len(twitter) * 100

print('Ответ: процент удалённых твитов -', deleted_percent, 'процентов.')

Ответ: процент удалённых твитов - 14.162754303599373 процентов.


Функция принимает на вход частотный словарь и число, задающее количество элементов, которое нужно вывести. Функция превращает словарь в отсортированный список из кортежей, расположенных в порядке убывания по частоте, и выводит заданное количество первых элементов. 

In [6]:
def sort_dict_max(all_dict, number):
    all_dict_sorted = sorted(all_dict.items(), key=lambda x:
                             (x[1], x[0]), reverse=True)
    counter = 0
    for item in all_dict_sorted:
        print(item[0], ' : ', item[1])
        counter += 1
        if counter >= number:
            break

# Пункт 3

Чтобы найти топ-16 самых популярных языков твитов, создаю словарь и прохожу по каждому твиту (ключ - "lang"), считывая его язык. Если в словаре ещё не существует элемент с таким ключом (язык твита), то создаю новый элемент с языком твита в ключе и количеством употребления в значении. Увеличиваю значение элемента на 1. Затем применяю функцию sort_dict_max(), чтобы найти топ-16 языков твитов.

In [7]:
tweet_lang = {}

for line in twitter:
    keys = list(line.keys())
    if keys[0] != 'delete':  # проверяю твит на существование
        tweet_lang.setdefault(line['lang'], 0)
        tweet_lang[line['lang']] += 1

print('Ответ: топ-16 языков твитов:')
sort_dict_max(tweet_lang, 16)

Ответ: топ-16 языков твитов:
en  :  719
ja  :  438
es  :  173
ko  :  149
th  :  123
ar  :  119
und  :  117
in  :  71
pt  :  69
fr  :  35
tr  :  30
tl  :  29
hi  :  23
ru  :  15
zh  :  8
fa  :  8


# Пункт 4

Чтобы найти количество пользователей, которым принадлежат несколько твитов из набора (ключ - "id_str"), создаю словарь, в котором лежат элементы с id пользователя в ключе и с количеством встретившихся твитов этого пользователя в значении. Алгоритм такой же, как в пункте 3. Затем прохожу по словарю и складываю количество пользователей, имеющих больше одного твита в наборе.

In [8]:
users = {}

for line in twitter:
    keys = list(line.keys())
    if keys[0] != 'delete':  # проверяю твит на существование
        user_id = line['user']['id_str']
        users.setdefault(user_id, 0)
        users[user_id] += 1

users_counter = 0

for number in list(users.values()):
    if number > 1:
        users_counter += 1

print('Ответ: количество пользователей с несколькими твитами в наборе -',
      users_counter, 'пользователей.')

Ответ: количество пользователей с несколькими твитами в наборе - 25 пользователей.


# Пункт 5

Чтобы найти топ-20 хэштегов, создаю список элементов, в который добавляю каждый хэштег, находящийся под ключом "text", который, в свою очередь, находится под ключами "entities" и "hashtags". Затем с помощью Counter создаю частотный словарь. Далее применяю функцию sort_dict_max(), чтобы найти топ-20 хэштегов.

In [9]:
from collections import Counter

hashtags_list = []

for line in twitter:
    keys = list(line.keys())
    if keys[0] != 'delete':  # проверяю твит на существование
        for hashtag in line['entities']['hashtags']:
            hashtags_list.append(hashtag['text'])

hashtags = Counter(hashtags_list)

print('Ответ: топ-20 хэштегов:')
sort_dict_max(hashtags, 20)

Ответ: топ-20 хэштегов:
BTS  :  17
방탄소년단  :  13
AMAs  :  11
人気投票ガチャ  :  8
태형  :  7
뷔  :  6
오늘의방탄  :  5
PledgeForSwachhBharat  :  5
MPN  :  5
BTSinChicago  :  5
BTSLoveYourselfTour  :  5
시카고1회차공연  :  4
เป๊กผลิตโชค  :  4
V  :  4
PCAs  :  4
JIMIN  :  4
지민  :  3
running  :  3
WajahmuPlastik  :  3
NCT127  :  3


# Пункт 6

Чтобы составить частотный словарь слов оригинальных твитов на английском языке, построчно прохожу по файлу и ищу твиты, которые не имеют ключи "retweeted_status" и "qouted_status" (так как они свидетельствую о неоригинальности твита), а также "delete", но языком которых является английский. Вынимаю текст твита по ключу "text" и очищаю его от знаков препинания, привожу к нижнему регистру. Разбиваю очищенный текст на слова и состовляю частотный словарь, элементы которого отсортировываю функцией sort_dict_max().

In [17]:
import string

words = []

# символы, от которых надо очистить текст
punctuation_extended = list(string.punctuation) + ['…', '°']
punctuation_extended.extend('0123456789')

for line in twitter:
    keys = list(line.keys())
    if ('retweeted_status' not in keys and 'quoted_status' not in keys and
            keys[0] != 'delete'):
        if line['lang'] == 'en':
            tweet_text = line['text'].lower()
            for char in punctuation_extended:
                tweet_text = tweet_text.replace(char, '')
            tweet_list = tweet_text.split()
            words.extend(tweet_list)

words_counter = Counter(words)

print('Ответ: топ-20 слов из частотного словаря ориг. твитов на англ. языке.:')
sort_dict_max(words_counter, 20)

Ответ: топ-20 слов из частотного словаря ориг. твитов на англ. языке.:
the  :  107
to  :  79
a  :  68
i  :  60
and  :  58
you  :  45
of  :  41
is  :  41
for  :  40
it  :  38
in  :  32
that  :  30
my  :  26
on  :  25
me  :  25
be  :  22
this  :  20
are  :  20
have  :  19
so  :  18


# Пункт 7

Чтобы найти топ-10 авторов твитов с наибольшим количеством подписчиков, создаю словарь с элементами, ключом которых является id пользователя, а значением - количество его подписчиков. Затем применяю функцию sort_dict_max() и вывожу топ-10 пользователей.

In [15]:
followers_total = {}

for line in twitter:
    keys = list(line.keys())
    if keys[0] != 'delete':  # проверяю твит на существование
        user_id = line['user']['id_str']
        followers_count = line['user']['followers_count']
        followers_total[user_id] = followers_count

print('Ответ: топ-10 пользоват. с наибольшим кол-вом подписчиков \
      (id : кол-во):')
sort_dict_max(followers_total, 10)

Ответ: топ-10 пользоват. с наибольшим кол-вом подписчиков       (id : кол-во):
262389559  :  2521403
14562016  :  1491309
18040230  :  1206759
1435461  :  1137374
290401936  :  625463
234419133  :  392472
20689749  :  383698
223282435  :  374222
634594307  :  318189
399622781  :  311319


# Пункт 8

Чтобы найти топ-10 источников твита, прохожу по строкам с твитами и по ключу "source" нахожу источник твита, который нужно вынуть с помощью регулярных выражений. Из списка составляю частотный словарь, который потом обрабатываю функцией sort_dict_max()

In [16]:
import re

sources = []

for line in twitter:
    keys = list(line.keys())
    if keys[0] != 'delete':  # проверяю твит на существование
        source = line['source']
        source_bare = re.search('rel="nofollow">([^<]*)</a>', source)
        sources.append(source_bare.group(1))

sources_counter = Counter(sources)

print('Ответ: топ-10 источников твита:')
sort_dict_max(sources_counter, 10)

Ответ: топ-10 источников твита:
Twitter for iPhone  :  800
Twitter for Android  :  695
Twitter Web Client  :  140
twittbot.net  :  122
Twitter Lite  :  51
Twitter for iPad  :  28
TweetDeck  :  23
Facebook  :  17
IFTTT  :  14
تطبيق قرآني  :  10
